<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/24_Neural_Network_Batch_Processing_Pass_Image_Batch_To_PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Batch Processing With PyTorch
在本节中的目标是将一批图像传递到我们的网络并解释结果。 事不宜迟，让我们开始吧。

* 准备数据

* 建立模型

  * 了解批处理如何传递到网络

* 训练模型

* 分析模型的结果

在上一节中，我们了解了前向传播以及如何将单个图像从我们的训练集中传递到我们的网络。 现在，让我们看看如何使用一批图像来完成此操作。 我们将使用数据加载器来获取批处理，然后将批处理传递到网络之后，我们将解释输出。




In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
    self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
    self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
    self.fc2 = nn.Linear(in_features=120,out_features=60)
    self.out = nn.Linear(in_features=60,out_features=10)
  
  def forward(self,t):
      t = t;

      t = self.conv1(t)
      t = F.relu(t)
      t = F.max_pool2d(t,kernel_size=2,stride=2)

      t = self.conv2(t)
      t = F.relu(t)
      t = F.max_pool2d(t,kernel_size=2,stride=2)
      
      t = t.reshape(-1,12*4*4)
      t = self.fc1(t)
      t = F.relu(t)

      t = self.fc2(t)
      t = F.relu(t)
      
      t = self.out(t)
      return t;

In [ ]:
train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,
                                download=True,
                                transform=transforms.Compose([
        transforms.ToTensor()
    ]))


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


## 1.将一批图像传递到网络

让我们首先回顾上一节的代码设置。 我们需要以下内容：

* 导入库。

* 导入训练集。

* 定义神经网络类。

* 禁用计算图梯度下降。 （可选的）

* 创建网络类实例。

现在，我们将使用我们的训练集来创建一个新的DataLoader实例，并设置我们的batch_size = 10，这样输出将更易于管理。

In [ ]:
data_loader = torch.utils.data.DataLoader(train_set,batch_size=10)

我们将从数据加载器中提取一批，并从该批中解压图像和标签张量。我们将使用复数形式来命名变量，因为我们知道在调用数据加载器迭代器的next时，数据加载器将返回一批10个图像。

In [ ]:
batch = next(iter(data_loader))
images,labels = batch

这给了我们两个张量，一个图像张量和一个对应标签的张量。

在上一集中，当我们从训练集中提取单个图像时，我们不得不unsqueeze()张量以添加另一个维度，该维度将有效地将单例图像转换为一个大小为1的批处理。 现在，我们正在使用数据加载器，默认情况下，我们正在处理批处理，因此不需要进一步的处理。

数据加载器返回一批图像，这些图像打包成单个张量，其形状可反映以下轴。
* (batch size, input channels, height, width)

这意味着张量的形状很好，不需要unsquzze()它。;)

In [ ]:
images.shape

torch.Size([10, 1, 28, 28])

In [ ]:
labels.shape

torch.Size([10])

让我们解释这两种形状。 图像张量的第一个轴告诉我们，我们有一批十张图像。 这十个图像具有一个高度和宽度为28的单一颜色通道。

标签张量的单轴形状为十，对应于我们批中的十个图像。 每个图像一个标签。

好吧。 让我们通过将图像张量传递到网络来进行预测。

In [ ]:
network= Network()
preds = network(images)

In [ ]:
preds.shape

torch.Size([10, 10])

In [ ]:
preds

tensor([[ 0.0155,  0.1265,  0.1793, -0.0976, -0.0550, -0.0595, -0.0050, -0.0247,
         -0.0926,  0.0896],
        [ 0.0202,  0.1256,  0.1868, -0.0995, -0.0494, -0.0613, -0.0055, -0.0162,
         -0.0950,  0.0911],
        [ 0.0165,  0.1316,  0.1712, -0.0981, -0.0662, -0.0479, -0.0065, -0.0381,
         -0.0845,  0.0773],
        [ 0.0145,  0.1263,  0.1743, -0.1011, -0.0601, -0.0539, -0.0035, -0.0277,
         -0.0891,  0.0790],
        [ 0.0129,  0.1274,  0.1907, -0.0951, -0.0524, -0.0586, -0.0051, -0.0144,
         -0.0929,  0.0853],
        [ 0.0218,  0.1309,  0.1851, -0.1027, -0.0466, -0.0635, -0.0024, -0.0130,
         -0.0964,  0.0921],
        [ 0.0206,  0.1299,  0.1736, -0.1010, -0.0577, -0.0554, -0.0049, -0.0318,
         -0.0893,  0.0830],
        [ 0.0232,  0.1288,  0.1815, -0.0948, -0.0435, -0.0721, -0.0018, -0.0188,
         -0.1023,  0.0935],
        [ 0.0129,  0.1252,  0.1705, -0.0968, -0.0739, -0.0421, -0.0033, -0.0486,
         -0.0814,  0.0861],
        [ 0.0129,  

预测张量的形状为10 x 10，这使我们得到两个轴，每个轴的长度为10。 这反映了以下事实：我们有十个图像，并且对于这十个图像中的每一个，我们都有十个预测类别。也即
* (batch size, number of prediction classes)

第一维的元素是长度为十的数组。 这些数组元素中的每个元素都包含对应图像每个类别的十个预测。

第二维的元素是数字。 每个数字都是特定输出类别的分配值。 输出类由索引编码，因此每个索引代表一个特定的输出类。 该映射由该表给出。

* Fashion MNIST Classes
* 0	T-shirt/top
* 1	Trouser
* 2	Pullover
* 3	Dress
* 4	Coat
* 5	Sandal
* 6	Shirt
* 7	Sneaker
* 8	Bag
* 9	Ankle boot

## 2. 使用 Argmax: 预测 Vs 标签

为了对照标签检查预测，我们使用argmax（）函数找出哪个索引包含最高的预测值。 一旦知道哪个索引具有最高的预测值，就可以将索引与标签进行比较，以查看是否存在匹配项。

为此，我们在预测张量上调用argmax（）函数，并指定第二维。

第二个维度是我们的预测张量的最后一个维度。 请记住，在我们所有关于张量的工作中，张量的最后一个维度始终包含数字，而其他所有维度都包含其他较小的张量。

在我们的预测张量的情况下，我们有十组数字。 argmax（）函数的作用是查看这十组中的每组，找到最大值，然后输出其索引。

对于每组十个数字：

* 查找最大值。

* 输出最大值对应的索引

对此的解释是，对于批次中的每个图像，我们正在找到具有最高值的预测类别。 这是网络最能预测的类别。

In [ ]:
preds.argmax(dim=1)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
labels

tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])

argmax（）函数的结果是十个预测类别的张量。 每个数字都是出现最大值的索引。 我们有十个数字，因为有十个图像。 一旦有了这个具有最高值的索引张量，就可以将其与标签张量进行比较。

In [ ]:
preds.argmax(dim=1).eq(labels)

tensor([False, False, False, False, False,  True, False,  True, False, False])

In [ ]:
preds.argmax(dim=1).eq(labels).sum()

tensor(2)

为了实现比较，我们使用eq（）函数。 eq（）函数计算argmax输出和标签张量之间的逐元素相等运算。

如果argmax输出中的预测类别与标签匹配，则返回True，否则返回False。

最后，如果在此结果上调用sum（）函数，则可以将输出减少为该标量值张量内的单个正确预测数。

我们可以将最后一个函数包装到名为get_num_correct（）的函数中，该函数接受预测和标签，并使用item（）方法返回Python数目的正确预测。

In [ ]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

调用这个函数，我们可以看到我们得到的值

In [ ]:
get_num_correct(preds,labels)

2

## 3.总结
我们现在应该很好地理解如何将一批输入传递给一个网络，以及在处理卷积神经网络时期望的形状是什么。